In [3]:
import datetime
import json
import rdflib
import requests
import os
from collections import OrderedDict
start = datetime.datetime.utcnow()
LDP = rdflib.Namespace('http://www.w3.org/ns/ldp#')

In [26]:
print(start)

2019-09-11 14:50:58.576005


## 2019-09-11 14:50:58.57600
# Restoring references RTs

In [4]:
walker = os.walk("/Users/jpnelson/2019/sinopia-data/2019/09/09/resource_templates/")

In [5]:
result = next(walker)

In [6]:
backup_rts = result[2]

In [12]:
result[0]

'/Users/jpnelson/2019/sinopia-data/2019/09/09/resource_templates/'

In [7]:
resource_templates = []
for filename in backup_rts:
    filepath = f"{result[0]}{filename}"
    with open(filepath) as fo:
        resource_templates.append(json.load(fo))
        

In [16]:
len(resource_templates)

1081

In [8]:
rt_lookup ={}
for i,row in enumerate(resource_templates):
    rt_lookup[row['id']] = { "ident": i, "missing": False, "subjects": 0 }
    


In [48]:
def check_missing(rt_dict):
    start = datetime.datetime.utcnow()
    print(f"Start at {start}")
    stage_ld4p_url = "https://trellis.stage.sinopia.io/repository/ld4p/"
    for i,key in enumerate(rt_lookup.keys()):
        stage_url = f"{stage_ld4p_url}{key}"
        result = requests.get(stage_url)
        if result.status_code > 300:
            rt_lookup[key]["missing"] = True
        if not i%25:
            print(f"{i:,}", end="")
        else:
            print(".", end="")
    end = datetime.datetime.utcnow()
    print(f"Finished at {end}, total time {((end-start).seconds / 60.0):,}")
    

In [49]:
check_missing(rt_lookup)

Start at 2019-09-11 16:06:02.794589
0........................25........................50........................75........................100........................125........................150........................175........................200........................225........................250........................275........................300........................325........................350........................375........................400........................425........................450........................475........................500........................525........................550........................575........................600........................625........................650........................675........................700........................725........................750........................775........................800........................825........................850........................875.....................

In [54]:
with open("../../sinopia_acl/.cognitoToken") as fo:
    cognitoToken = fo.read()

headers = { 
    "Authorization": f"Bearer {cognitoToken}"
}

In [28]:
test_walker = next(os.walk("/Users/jpnelson/2019/sinopia-data/2019/09/10/test/"))
sinopia_rt_uri = rdflib.URIRef("http://sinopia.io/vocabulary/hasResourceTemplate")

In [43]:
def calculate_missing(dirpath, rt_dict):
    walker = next(os.walk(dirpath))
    for i,row in enumerate(walker[2]):
        filepath = os.path.join(walker[0], row)
        graph = rdflib.ConjunctiveGraph()
        graph.parse(filepath, format='turtle')
        for obj in graph.objects(predicate=sinopia_rt_uri):
            rt_id = str(obj)
            if not rt_id in rt_lookup.keys():
                print(f"{rt_id} not found", end=" ")
                continue
            if rt_lookup[rt_id].get('missing'):
                rt_lookup[rt_id]["subjects"] += 1
        if not i%25:
            print(f"{i:,}", end="")
        else:
            print(".", end="")

In [44]:
calculate_missing("/Users/jpnelson/2019/sinopia-data/2019/09/10/train/", rt_lookup)

0..................WAU:RT:RDA:Work:monograph not found ......25........................50.................

In [1]:
for key, val in rt_lookup.items():
    if key.startswith("Sinopia"):
        print(val)

NameError: name 'rt_lookup' is not defined

In [51]:
rt_lookup["sinopia:resourceTemplate:bf2:Item:Enumeration"]

{'ident': 8, 'missing': False, 'subjects': 0}

In [52]:
resource_templates[8]

{'author': 'NDMSO',
 'date': '2017-08-08',
 'id': 'sinopia:resourceTemplate:bf2:Item:Enumeration',
 'propertyTemplates': [{'mandatory': 'false',
   'propertyLabel': 'Enumeration',
   'propertyURI': 'http://www.w3.org/2000/01/rdf-schema#label',
   'repeatable': 'true',
   'type': 'literal'}],
 'resourceLabel': 'Enumeration',
 'resourceURI': 'http://id.loc.gov/ontologies/bibframe/Enumeration',
 'schema': 'https://ld4p.github.io/sinopia/schemas/0.1.0/resource-template.json'}

In [56]:
requests.put("https://trellis.stage.sinopia.io/repository/ld4p/sinopia:resourceTemplate:bf2:Item:Enumeration",
               data=resource_templates[8],
               headers=headers)

<Response [204]>

In [57]:
for key, val in rt_lookup.items():
    if val.get("missing") and val.get("subjects") > 0:
        sinopia_url = f"https://trellis.stage.sinopia.io/repository/ld4p/{key}"
        restore_result = requests.put(sinopia_url,
                                      data=resource_templates[val.get("ident")],
                                      headers=headers)
        if restore_result.status_code > 300:
            print(f"failed to restore {key}")

In [10]:
for key in rt_lookup.keys():
    if key.startswith("Sinopia"):
        print(key)

Sinopia:RT:Demo:FrickWork
Sinopia:RT:Demo:SampleResourceFuller
